In [0]:
import requests, json, time
from datetime import datetime, timezone
import base64
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

b64 = w.secrets.get_secret(scope="DatabricksSecretScope", key="METUA").value
decoded_ua = base64.b64decode(b64).decode("utf-8")

In [0]:
# --- API setup ---
UA = decoded_ua
BASE = "https://api.met.no/weatherapi/airqualityforecast/0.1/stations"
HEADERS = {"User-Agent": UA, "Accept": "application/json"}

In [0]:
# --- Target Volume path ---
# Databricks notebook parameter (from DAB)
dbutils.widgets.text("catalog", "main_uc_dev")
catalog = dbutils.widgets.get("catalog")

volume_root = f"/Volumes/{catalog}/bronze/met_airquality_stations"
target_dir  = volume_root
target_file = f"{target_dir}/airquality.jsonl"


In [0]:

# --- 4) Fetch all stations ---
records = []


r = requests.get(BASE, headers=HEADERS, timeout=20)
r.raise_for_status()
js = r.json()
time.sleep(0.2)


In [0]:

# Convert to JSON Lines text
jsonl_str = "\n".join(json.dumps(row, ensure_ascii=False) for row in js)

# Ensure directory and write the single JSONL file

dbutils.fs.mkdirs(target_dir)
dbutils.fs.put(target_file, jsonl_str, overwrite=True)



In [0]:
df = spark.read.format("json").load(target_file)
display(df)